In [1]:
import os
import pandas as pd
import geopandas as gpd
import json
import numpy as np
import folium
from folium.plugins import MarkerCluster, FloatImage, HeatMap

In [2]:
vci_df = pd.read_csv(os.path.join("data", "vci_constituency.csv"))

In [3]:
vci_df.head()

,constituency_name,counstituency_code,vci_aug_16,vci_sep_16,vci_nov_16,vci_dec_16,vci_jan_17
0,GANZE,15,11.85,14.66,2.80,-10.82,2.64
1,KILIFI NORTH,11,12.36,16.56,-0.08,-23.20,-13.79
2,KALOLENI,13,14.14,14.65,-6.57,-23.91,-15.14
3,MALINDI,16,16.65,16.16,-1.17,-16.81,-7.23
4,KINANGO,10,19.17,17.62,3.52,-4.82,3.68


In [4]:
january_vci = {k: v for (k, v) in zip([row.counstituency_code for index, row in vci_df.iterrows()],
                                     [row.vci_jan_17 for index, row in vci_df.iterrows()])}

In [5]:
constituencies_gdf = gpd.GeoDataFrame.from_file(os.path.join("data", "constituencies.geojson"))
constituencies_gdf.head()

,CONSTITUEN,CONST_CODE,geometry
0,KAMUKUNJI,288.0,POLYGON ((36.87595372679757 -1.262533413477499...
1,WESTLANDS,274.0,POLYGON ((36.81926300479753 -1.206434634477475...
2,DAGORETTI NORTH,275.0,POLYGON ((36.79214315479756 -1.260847108477478...
3,ROYSAMBU,279.0,POLYGON ((36.90578043879756 -1.161375592477484...
4,STAREHE,289.0,POLYGON ((36.84274500879754 -1.264599326477466...


In [6]:
styles = []

for row in constituencies_gdf["CONST_CODE"]:
    try:
        vci_value = january_vci[row]
        if vci_value >= 50:
            styles.append({"fillColor": "#0A9500", "weight": 1, "color": "#000000"})
        elif vci_value >= 35:
            styles.append({"fillColor": "#79C573", "weight": 1, "color": "#000000"})
        elif vci_value >= 21:
            styles.append({"fillColor": "#FFD819", "weight": 1, "color": "#000000"})
        elif vci_value >= 10:
            styles.append({"fillColor": "#FF8B8B", "weight": 1, "color": "#000000"})
        else:
            styles.append({"fillColor": "#FF0000", "weight": 1, "color": "#000000"})
    except KeyError:
        styles.append({"fillColor": "#424242", "weight": 1, "color": "#000000"})

constituencies_gdf["style"] = styles
constituencies_gdf.head(2)

,CONSTITUEN,CONST_CODE,geometry,style
0,KAMUKUNJI,288.0,POLYGON ((36.87595372679757 -1.262533413477499...,"{'fillColor': '#424242', 'color': '#000000', '..."
1,WESTLANDS,274.0,POLYGON ((36.81926300479753 -1.206434634477475...,"{'fillColor': '#424242', 'color': '#000000', '..."


In [8]:
vci_map = folium.Map([
    np.mean([row.geometry.centroid.y for index, row in constituencies_gdf.iterrows()]), 
    np.mean([row.geometry.centroid.x for index, row in constituencies_gdf.iterrows()])], 
    zoom_start = 6,
    tiles = None)

folium.GeoJson(constituencies_gdf).add_to(vci_map)

FloatImage("legend.png", bottom = 30, left = 75).add_to(vci_map)

vci_map.save(os.path.join("html_files", "constituencies_no_tiles.html"))

In [9]:
vci_map = folium.Map([
    np.mean([row.geometry.centroid.y for index, row in constituencies_gdf.iterrows()]), 
    np.mean([row.geometry.centroid.x for index, row in constituencies_gdf.iterrows()])], 
    zoom_start = 6,
    tiles = "Cartodb Positron")

folium.GeoJson(constituencies_gdf).add_to(vci_map)

FloatImage("legend.png", bottom = 30, left = 75).add_to(vci_map)

vci_map.save(os.path.join("html_files", "constituencies_without_markers.html"))

In [10]:
vci_map = folium.Map([
    np.mean([row.geometry.centroid.y for index, row in constituencies_gdf.iterrows()]), 
    np.mean([row.geometry.centroid.x for index, row in constituencies_gdf.iterrows()])], 
    zoom_start = 6,
    tiles = "Cartodb Positron")

folium.GeoJson(constituencies_gdf).add_to(vci_map)

FloatImage("legend.png", bottom = 30, left = 75).add_to(vci_map)

vci_map.add_child(MarkerCluster(locations=list(zip([row.geometry.centroid.y for index, row in constituencies_gdf.iterrows()], 
                                                   [row.geometry.centroid.x  for index, row in constituencies_gdf.iterrows()])), 
                                popups=[row.CONSTITUEN for index, row in constituencies_gdf.iterrows()]))

vci_map.save(os.path.join("html_files", "constituencies_with_markers.html"))

In [11]:
parks_df = pd.read_csv(os.path.join("data", "parks_reserves.csv"))

In [12]:
vci_map = folium.Map([
    np.mean([row.geometry.centroid.y for index, row in constituencies_gdf.iterrows()]), 
    np.mean([row.geometry.centroid.x for index, row in constituencies_gdf.iterrows()])], 
    zoom_start = 6,
    tiles = "Cartodb Positron")

folium.GeoJson(constituencies_gdf).add_to(vci_map)

FloatImage("legend.png", bottom = 30, left = 75).add_to(vci_map)

vci_map.add_child(MarkerCluster(locations=list(zip([row.lat for index, row in parks_df.iterrows()], 
                                                  [row.lon  for index, row in parks_df.iterrows()])), 
                               popups=[row["name"] for index, row in parks_df.iterrows()]))

HeatMap(list(zip([row.lat for index, row in parks_df.iterrows()], 
                 [row.lon  for index, row in parks_df.iterrows()]))).add_to(vci_map)

vci_map.save(os.path.join("html_files", "constituencies_with_reserves_heatmap.html"))

In [14]:
vci_map = folium.Map([
    np.mean([row.geometry.centroid.y for index, row in constituencies_gdf.iterrows()]), 
    np.mean([row.geometry.centroid.x for index, row in constituencies_gdf.iterrows()])], 
    zoom_start = 6,
    tiles = "Cartodb Positron")

folium.GeoJson(constituencies_gdf).add_to(vci_map)

#FloatImage("legend.png", bottom = 30, left = 75).add_to(vci_map)

#vci_map.add_child(MarkerCluster(locations=list(zip([row.lat for index, row in parks_df.iterrows()], 
#                                                  [row.lon  for index, row in parks_df.iterrows()])), 
#                               popups=[row["name"] for index, row in parks_df.iterrows()]))

HeatMap(list(zip([row.lat for index, row in parks_df.iterrows()], 
                 [row.lon  for index, row in parks_df.iterrows()]))).add_to(vci_map)

vci_map.save(os.path.join("html_files", "constituencies_with_reserves_heatmap_unclustered.html"))